In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
hansard_hong_kong_df = pd.read_csv('intermediate_outputs/hansard_hong_kong.csv')
scrape_results_df = pd.read_csv('intermediate_outputs/scrape_results.csv')

merged_df = pd.merge(
    hansard_hong_kong_df,
    scrape_results_df,
    how = 'left',
    left_on = 'gid',
    right_on = 'html_file_name',
    suffixes = ('', '_collasped')
)

In [3]:
merged_df = merged_df.drop_duplicates()

major_dict = {
    '1': 'Commons debates',
    '101': 'Lords debates',
    '2': 'Westminster Hall debates',
    '3': 'Written Answers',
    '4': 'Written Ministerial Statements',
    '5': 'Northern Ireland Assembly debates',
    '6': 'Public Bill Committees',
    '7': 'Scottish Parliament debates',
    '8': 'Scottish Parliament written answers',
    '9': 'Questions to the Mayor of London',
    '10': 'Welsh Parliament record',
    '11': 'Senedd Cymru Cofnod'
}
merged_df['debate_type'] = merged_df['major'].map(major_dict)
merged_df['debate_type'].fillna('Other', inplace=True)

minor_dict = {
    0: 'Not applicable',
    1: 'Question',
    2: 'Answer'
}
merged_df['written_type'] = merged_df['minor'].map(minor_dict)
merged_df['written_type'].fillna('Other', inplace=True)

def extract_text_from_html(html_content):
    if pd.notna(html_content) and isinstance(html_content, str):
        soup = BeautifulSoup(html_content, 'html.parser')
        return soup.get_text()
    else:
        return ''

merged_df['body'] = merged_df['body'].apply(extract_text_from_html)
merged_df['speech_body'] = np.where(
    merged_df['collapsed'] == 0,
    merged_df['body'],
    merged_df['body_collasped']
)
merged_df = merged_df[(merged_df['speech_body'] != '') & (merged_df['speech_body'].notna())]

merged_df['full_url'] = 'https://www.theyworkforyou.com/' + merged_df['listurl']

merged_df['relevant_speeches'] = merged_df['collapsed'] + 1

merged_df['speaker_id'] = np.where(
    merged_df['collapsed'] == 0,
    merged_df['person_id'],
    merged_df['person_id_collasped']
)
merged_df['speaker_id'] = merged_df['speaker_id'].str.replace('Not found', 'Not found/Not an MP')
merged_df['speaker_id'] = np.where(
    (merged_df['speaker_id'].str.match(r'^\d{1,4}$')) | (merged_df['speaker_id'].str.contains('wikipedia')),
    'Not found/Not an MP',
    merged_df['speaker_id']
)

merged_df = merged_df.drop(['htime', 'section_id', 'subsection_id', 'htype', 'major', 'minor', 'person_id', 'hpos', 'epobject_id', 'body', 'collapsed', 'relevance', 'extract', 'listurl', 'speaker_member_id', 'speaker_name', 'speaker_house', 'speaker_constituency', 'speaker_party', 'speaker_person_id', 'speaker_url', 'speaker_office', 'parent_listurl', 'person_id_collasped', 'body_collasped'], axis = 1)

In [ ]:
merged_df.to_csv('intermediate_outputs/all_speeches.csv', index=False)